# ⚽ Análisis de Estadísticas de Partidos de Fútbol con PySpark

Este notebook aplica técnicas de PySpark (DataFrames, Agregaciones, UDFs y Funciones de Ventana) para el análisis de datos de fútbol.

In [8]:
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Window
from math import radians, cos, sin, asin, sqrt

# --- REEMPLAZO IMPORTANTE: Montar Google Drive ---
# Ejecuta esto si tus archivos están en Drive. Reemplaza 'path_to_data' por la ruta real si usas el método de enlace simbólico.
from google.colab import drive
drive.mount('/content/drive')

# Definir la carpeta base donde se encuentran los CSVs (AJUSTAR SEGÚN TU RUTA DE DRIVE)
# Ejemplo: Si los datos están en 'Mi Drive/Colab Notebooks/data/soccer-data/'
DATA_BASE_PATH = '/content/drive/MyDrive/Colab Notebooks/soccer-data/'

# Creación de la sesión de Spark
spark = (SparkSession.builder
    .appName("Soccer Analytics")
    .getOrCreate())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Ejecutar para instalar OpenJDK 21 si el paso anterior falla
!apt install openjdk-21-jdk

---

## 🏆 Dataset 1: Resultados de Partidos (`results.csv`)

### Carga y Exploración Inicial

In [9]:
# Definición del esquema para resultados de partidos (para evitar inferencia lenta/errónea)
resultsSchema = StructType([
    StructField("date", DateType(), False),
    StructField("home_team", StringType(), False),
    StructField("away_team", StringType(), False),
    StructField("home_score", IntegerType(), False),
    StructField("away_score", IntegerType(), False),
    StructField("tournament", StringType(), False),
    StructField("city", StringType(), False),
    StructField("country", StringType(), False),
    StructField("neutral", StringType(), False)
])

# Carga del DataFrame de Resultados
results_df = (spark.read.option("header", "true")
              .option("dateFormat", "yyyy-MM-dd")
              .csv(DATA_BASE_PATH + "results.csv", schema=resultsSchema)
             )

results_df.printSchema()
results_df.show(5)
print(f"Número total de partidos: {results_df.count()}")

root
 |-- date: date (nullable = true)
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- home_score: integer (nullable = true)
 |-- away_score: integer (nullable = true)
 |-- tournament: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- neutral: string (nullable = true)

+----------+---------------+-----------------+----------+----------+--------------+----------+-------+-------+
|      date|      home_team|        away_team|home_score|away_score|    tournament|      city|country|neutral|
+----------+---------------+-----------------+----------+----------+--------------+----------+-------+-------+
|2023-08-12| Manchester Utd|           Wolves|         1|         0|Premier League|Manchester|England|  FALSE|
|2023-08-12|        Arsenal|Nottingham Forest|         2|         1|Premier League|    London|England|  FALSE|
|2023-08-12|        Chelsea|        Liverpool|         1|         1|Premier Leagu

### Ejercicio 1: Resumen por Torneo

Calcule para cada **Torneo** la fecha del primer partido (`start_date`), la fecha del último partido (`last_date`) y el **número total de goles** marcados (`total_goals`) en dicho torneo.

In [15]:
# Solución Ejercicio 1

---

## 🥅 Dataset 2: Goleadores por Partido (`goalscorers.csv`)

### Carga y Exploración Inicial

In [11]:
# Definición del esquema para goleadores
scorersSchema = StructType([
    StructField("date", DateType(), False),
    StructField("home_team", StringType(), False),
    StructField("away_team", StringType(), False),
    StructField("team", StringType(), False),
    StructField("scorer", StringType(), False),
    StructField("minute", IntegerType(), True),
    StructField("own_goal", StringType(), True),
    StructField("penalty", StringType(), True)
])

# Carga del DataFrame de Goleadores
scorers_df = (spark.read.option("header", "true")
              .option("dateFormat", "yyyy-MM-dd")
              .csv(DATA_BASE_PATH + "goalscorers.csv", schema=scorersSchema)
             )

scorers_df.printSchema()
scorers_df.show(5, truncate=False)
print(f"Número total de goles registrados: {scorers_df.count()}")

root
 |-- date: date (nullable = true)
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- team: string (nullable = true)
 |-- scorer: string (nullable = true)
 |-- minute: integer (nullable = true)
 |-- own_goal: string (nullable = true)
 |-- penalty: string (nullable = true)

+----------+--------------+-----------------+-----------------+-----------+------+--------+-------+
|date      |home_team     |away_team        |team             |scorer     |minute|own_goal|penalty|
+----------+--------------+-----------------+-----------------+-----------+------+--------+-------+
|2023-08-12|Manchester Utd|Wolves           |Manchester Utd   |Rashford M.|15    |NULL    |NULL   |
|2023-08-12|Arsenal       |Nottingham Forest|Arsenal          |Saka B.    |8     |NULL    |NULL   |
|2023-08-12|Arsenal       |Nottingham Forest|Nottingham Forest|Awoniyi T. |42    |NULL    |NULL   |
|2023-08-12|Arsenal       |Nottingham Forest|Arsenal          |Odegaard M.|78    |NULL

### Ejercicio 2: Ranking de Goleadores (Agregación)

Calcule el **número total de goles** y la **media de minuto** en el que marca el gol para cada jugador (`scorer`). Muestre el ranking de los 5 mejores goleadores.

In [16]:
# Solución Ejercicio 2

---

### Ejercicio 3: Cálculo de Valor de Gol con UDF

Cree una función de usuario (UDF) llamada `goal_value` que asigne un **valor** a cada gol en función del minuto:

- Gol en el minuto 90 o posterior: **Valor 3** (Gol tardío/decisivo)
- Gol entre el minuto 76 y 89: **Valor 2**
- Gol antes del minuto 76: **Valor 1**

Calcule el valor total de los goles para cada jugador.

#### 💡 ¿Qué es una UDF (User Defined Function)?

Una **UDF** (Función Definida por el Usuario) es una función personalizada que permite extender las capacidades de PySpark aplicando lógica personalizada a los datos.

**Características principales:**
- ✅ Permite aplicar funciones Python personalizadas sobre columnas de DataFrames
- ✅ Útil cuando las funciones nativas de Spark no cubren una necesidad específica
- ✅ Se puede aplicar usando `.withColumn()` o `.select()`
- ⚠️ **Nota de rendimiento:** Las UDF son más lentas que las funciones nativas de Spark porque requieren serialización/deserialización de datos entre la JVM y Python

**Sintaxis básica:**
```python
from pyspark.sql.functions import udf
from pyspark.sql.types import TipoRetorno

# 1. Definir función Python
def mi_funcion(valor):
    return resultado

# 2. Convertir a UDF de Spark
mi_udf = udf(mi_funcion, TipoRetorno())

# 3. Aplicar la UDF
df = df.withColumn("nueva_columna", mi_udf(col("columna_existente")))
```

In [17]:
# Solución Ejercicio 3

---

### Ejercicio 4: Goles Acumulados por Equipo (Funciones de Ventana)

Utilizando las **Funciones de Ventana** (`Window`), calcule el **número acumulado de goles** para cada equipo (`team`), ordenados por la fecha (`date`) y el minuto (`minute`) del gol. Esto simula el *running total* de la evolución goleadora.

#### 💡 ¿Qué son las Funciones de Ventana (Window Functions)?

Las **Funciones de Ventana** permiten realizar cálculos sobre un conjunto de filas relacionadas con la fila actual, sin colapsar los resultados en una sola fila (como lo haría `groupBy`).

**Características principales:**
- ✅ Mantienen todas las filas originales en el resultado
- ✅ Permiten realizar cálculos como **sumas acumuladas**, **rankings**, **promedios móviles**, etc.
- ✅ Se definen mediante **particiones** (grupos) y **ordenamiento**
- 🎯 Muy útiles para análisis de series temporales y rankings

**Componentes de una ventana:**
1. **`partitionBy()`**: Divide los datos en grupos (similar a `groupBy`)
2. **`orderBy()`**: Define el orden dentro de cada partición
3. **`rowsBetween()` / `rangeBetween()`**: Define el rango de filas a considerar (opcional)

**Sintaxis básica:**
```python
from pyspark.sql import Window
from pyspark.sql.functions import sum, rank, row_number

# Definir la ventana
window_spec = Window.partitionBy('columna_grupo').orderBy('columna_orden')

# Aplicar función de ventana
df = df.withColumn('suma_acumulada', sum('columna').over(window_spec))
df = df.withColumn('ranking', rank().over(window_spec))
```

**Funciones de ventana comunes:**
- `sum()`, `avg()`, `min()`, `max()`: Agregaciones sobre la ventana
- `rank()`, `dense_rank()`, `row_number()`: Rankings
- `lag()`, `lead()`: Acceso a filas anteriores/siguientes

---

#### 🔍 Pistas para el Ejercicio 4:

1. **Crear un identificador único**: Usa `monotonically_increasing_id()` para crear un ID de secuencia único por cada gol. Esto garantiza un orden determinista cuando hay múltiples goles en el mismo minuto.
   ```python
   df = df.withColumn("goal_sequence_id", monotonically_increasing_id())
   ```

2. **Ordenamiento de la ventana**: Para una suma acumulada correcta, ordena por `date` y luego por el `goal_sequence_id`:
   ```python
   window_spec = Window.partitionBy('team').orderBy('date', 'goal_sequence_id')
   ```

3. **Contar goles con `lit(1)`**: Para contar un gol por cada fila, usa `lit(1)` en lugar de contar una columna específica:
   ```python
   sum(lit(1)).over(window_spec)
   ```
   Esto suma el valor `1` por cada fila, generando así el conteo acumulado de goles.

In [18]:
# Solución Ejercicio 4